
# **YOLOv8.2-segment**

Это руководство основано на [репозитории ultralytics](ultralytics).
  
Тип разметки для YOLO - набор точек объектов

# **1. Установка зависимостей**

# 1.1 Монтирование Google Drive  
Можно не монтировать гугл диск, а хранить все данные для выполнения дз в файлах ноутбука, но тогда репозиторий YOLO, результаты обучения и датасет не сохранятся при отключении от среды выполнения. То есть каждый раз надо будет клонировать репозиторий и загружать датасет, что не очень удобно.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Определяем видеокарту GPU, чтобы на ней учить нейронную сеть

In [3]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


# 1.2 Установка зависимостей

In [4]:
%pip install ultralytics
import ultralytics
import os
import json
import shutil
import yaml
from sklearn.model_selection import train_test_split
ultralytics.checks()

Ultralytics YOLOv8.2.34 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.3/107.7 GB disk)


# **2. Импорт датасета**

Набор данных должен быть экспортирован из cvat в формате COCO 1.0.  

В бесплатной онлайн-версии нельзя экспортировать датасет вместе с картинками, если используется эта версия, картинки надо загружать отдельно. Если устанавливать cvat локально, такого ограничения нет.

Загрузите аннотации и картинки в одну папку и загрузите ее на google диск,туда же поместите файл instances_default.json.

In [5]:
def convert_coco_to_yolo_segmentation(json_file, folder_name = "labels"):
    folder_name = folder_name
    # Load the JSON file
    with open(json_file, 'r') as file:
        coco_data = json.load(file)

    # Create a "labels" folder to store YOLO segmentation annotations
    output_folder = os.path.join(os.path.dirname(json_file), folder_name)
    os.makedirs(output_folder, exist_ok=True)

    # Extract annotations from the COCO JSON data
    annotations = coco_data['annotations']
    for annotation in annotations:
        image_id = annotation['image_id']
        category_id = annotation['category_id']
        segmentation = annotation['segmentation']
        bbox = annotation['bbox']

        # Find the image filename from the COCO data
        for image in coco_data['images']:
            if image['id'] == image_id:
                image_filename = os.path.basename(image['file_name'])
                image_filename = os.path.splitext(image_filename)[0] # Removing the extension. (In our case, it is the .jpg or .png part.)
                image_width = image['width']
                image_height = image['height']
                break

        # Calculate the normalized center coordinates and width/height
        x_center = (bbox[0] + bbox[2] / 2) / image_width
        y_center = (bbox[1] + bbox[3] / 2) / image_height
        bbox_width = bbox[2] / image_width
        bbox_height = bbox[3] / image_height

        # Convert COCO segmentation to YOLO segmentation format
        yolo_segmentation = [f"{(x) / image_width:.5f} {(y) / image_height:.5f}" for x, y in zip(segmentation[0][::2], segmentation[0][1::2])]
        #yolo_segmentation.append(f"{(segmentation[0][0]) / image_width:.5f} {(segmentation[0][1]) / image_height:.5f}")
        yolo_segmentation = ' '.join(yolo_segmentation)

        # Generate the YOLO segmentation annotation line
        yolo_annotation = f"{category_id} {yolo_segmentation}"

        # Save the YOLO segmentation annotation in a file
        output_filename = os.path.join(output_folder, f"{image_filename}.txt")
        with open(output_filename, 'a+') as file:
            file.write(yolo_annotation + '\n')

    print("Conversion completed. YOLO segmentation annotations saved in 'labels' folder.")

# Example usage
json_file = "/content/gdrive/MyDrive/dataset/instances_default.json" #JSON file
split = "/content/gdrive/MyDrive/dataset" #Folder
convert_coco_to_yolo_segmentation(json_file, split)

Conversion completed. YOLO segmentation annotations saved in 'labels' folder.


In [6]:
dataset_path = '/content/gdrive/MyDrive/dataset' # Укажите свой путь до датасета

In [7]:
test_size = 0.2 # размер тестовой выборки
valid_size = 0.1 # размер валидационной выборки

In [8]:
image_files = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]
annotation_files = [f for f in os.listdir(dataset_path) if f.endswith('.txt')]

image_files_with_annotations = [f for f in image_files if f.replace('.jpg', '.txt') in annotation_files]

train_files, test_files = train_test_split(image_files_with_annotations, test_size=test_size, random_state=42)
train_files, valid_files = train_test_split(train_files, test_size=len(image_files_with_annotations)*valid_size/len(train_files), random_state=42)

def copy_files(files, source_path, dest_path, folder):
    for file in files:
        shutil.copy(os.path.join(source_path, file), os.path.join(dest_path, 'images', folder))
        shutil.copy(os.path.join(source_path, file.replace('.jpg', '.txt')), os.path.join(dest_path, 'labels', folder))

for folder in ['train', 'val', 'test']:
    os.makedirs(os.path.join(dataset_path, 'images', folder), exist_ok=True)
    os.makedirs(os.path.join(dataset_path, 'labels', folder), exist_ok=True)

copy_files(train_files, dataset_path, dataset_path, 'train')
copy_files(valid_files, dataset_path, dataset_path, 'val')
copy_files(test_files, dataset_path, dataset_path, 'test')

for file in os.listdir(dataset_path):
    if not file.endswith('.names') and os.path.isfile(os.path.join(dataset_path, file)):
      os.remove(os.path.join(dataset_path, file))

In [9]:
train_path = f'{dataset_path}/images/train'
val_path = f'{dataset_path}/images/val'
test_path = f'{dataset_path}/images/test'

# names_file = os.path.join(dataset_path, 'obj.names')
# with open(names_file, 'r') as file:
#     class_names = [line.strip() for line in file.readlines()]

# num_classes = len(class_names)

data = {
    'path': dataset_path,
    'train': train_path,
    'val': val_path,
    'test': test_path,
    'names': {
      0: 'airplane',
      1: 'weather_balloon',
      2: 'UFO'
    }
}

output_file = os.path.join(dataset_path, 'data.yaml')
with open(output_file, 'w') as file:
    documents = yaml.dump(data, file)

# os.remove(os.path.join(dataset_path, 'obj.names'))

# **3. Запуск обучения**

In [10]:
%cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


# 3.2 Начало обучения

[Чтобы получить полный список аргументов обучения, перейдите по ссылке](https://docs.ultralytics.com/modes/train/#train-settings)

In [13]:
# Load YOLOv8n-seg, train it on COCO128-seg for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')  # load a pretrained YOLOv8n segmentation model
model.train(data='dataset/data.yaml', epochs=10, imgsz=640, device=0)  # train the model
model('datasets_seg/images/train/airplane(0).jpeg')  # predict on an image

Ultralytics YOLOv8.2.34 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

100%|██████████| 755k/755k [00:00<00:00, 16.1MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/gdrive/MyDrive/dataset/labels/train... 158 images, 0 backgrounds, 0 corrupt: 100%|██████████| 158/158 [00:01<00:00, 83.60it/s] 


train: New cache created: /content/gdrive/MyDrive/dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/gdrive/MyDrive/dataset/labels/val... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<00:00, 102.00it/s]

val: New cache created: /content/gdrive/MyDrive/dataset/labels/val.cache


Plotting labels to runs/segment/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)


KeyboardInterrupt: 

# **4. Экспорт модели в onnx**

In [ ]:
!pip install ultralytics onnx onnx-simplifier onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.4 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# Load a YOLO-seg model
model = YOLO('runs/segment/train/weights/best.pt')

# Export to ONNX
onnx_file = model.export(format='onnx')

# Load an ONNX model
onnx_model = YOLO(onnx_file)

# Run inference on CPU and GPU with ONNXRuntime
results = onnx_model('dataset/images/test/airplane(11).jpg')

Ultralytics YOLOv8.2.27 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n-seg summary (fused): 195 layers, 3258649 parameters, 0 gradients, 12.0 GFLOPs

PyTorch: starting from 'runs/segment/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 39, 8400), (1, 32, 160, 160)) (6.5 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 1.5s, saved as 'runs/segment/train/weights/best.onnx' (12.6 MB)

Export complete (3.6s)
Results saved to /content/gdrive/MyDrive/runs/segment/train/weights
Predict:         yolo predict task=segment model=runs/segment/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=segment model=runs/segment/train/weights/best.onnx imgsz=640 data=datasets_seg/data.yaml  
Visualize:       https://netron.app
Loading runs/segment/train/weights/best.onnx for ONNX Runtime inference...

image 1/1 /content/gdrive/MyDrive/dataset/test/images/airplane(11).jpg: 640x640 1 airplane, 397.4ms
Spe

In [ ]:
print(results)

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'airplane', 1: 'weather_balloon', 2: 'UFO'}
obb: None
orig_img: array([[[184, 156, 132],
        [184, 156, 132],
        [185, 154, 131],
        ...,
        [159, 117,  88],
        [159, 117,  88],
        [159, 117,  88]],

       [[183, 155, 131],
        [183, 155, 131],
        [184, 153, 130],
        ...,
        [159, 117,  88],
        [159, 117,  88],
        [159, 117,  88]],

       [[182, 154, 130],
        [182, 154, 130],
        [183, 152, 129],
        ...,
        [158, 116,  87],
        [158, 116,  87],
        [158, 116,  87]],

       ...,

       [[165, 156, 152],
        [165, 156, 152],
        [165, 156, 152],
        ...,
        [171, 170, 174],
        [171, 169, 175],
        [172, 170, 176]],

       [[165, 156, 152],
        [166, 157, 153],
        [164, 158, 153],
      